In [163]:
import pandas as pd
import oracledb
import xml.etree.ElementTree as ET
import xmltodict
from deepdiff import DeepDiff
import json

In [164]:
def create_xml(protein_data):

    # Convert the protein_data list into a pandas DataFrame
    df = pd.DataFrame(protein_data, columns=[
        'protein_id', 'name', 'dbcode', 'crc64', 'length', 'timestamp', 'fragment', 'tax_id',
        'method_ac', 'model_ac', 'pos_from', 'pos_to', 'fragments', 'score', 'method_desc', 'status', 'dbname', 'evd'
    ])
    
    # Group by protein_id and method_ac, then create a nested dictionary
    grouped = {}

    # Iterate over each row and populate the nested dictionary
    for _, row in df.iterrows():

        protein_id = row['protein_id']

        if (not(protein_id in grouped.keys())):
            grouped[protein_id] = {
                "info": {
                "id": row["protein_id"],
                "name": row["name"],
                "length": row["length"],
                "crc64": row["crc64"],
                }
        }
            
    
        match_id = row['method_ac']
        location = {
            'start': row['pos_from'],
            'end': row['pos_to'],
            'fragments': row['fragments'],
            'score': int(float(row["score"])) if row["score"][-2:] == ".0" else row["score"]
        }

        print(location)

        if (match_id != ""):
            # Add the match_id and its location under the protein_id
            if (not(match_id in grouped[protein_id].keys())):
                grouped[protein_id][match_id] = {
                    "id": match_id,
                    "name": row["method_desc"],
                    "dbname": row["dbname"],
                    "status": row["status"],
                    "model": row["model_ac"],
                    "evd": row["evd"], 
                    "locations": [location]
                }
            else:
                grouped[protein_id][match_id]["locations"].append(location)

    

    # Create the root element for XML
    root = ET.Element("proteins")

    # Iterate through the grouped data to create XML structure
    for protein_id, protein_data in grouped.items():
        # Extract the info for the protein
        info = protein_data["info"]
        
        # Create a protein element
        protein_elem = ET.SubElement(root, "protein", 
                                     id=info["id"], 
                                     name=info["name"], 
                                     length=str(info["length"]), 
                                     crc64=info["crc64"])
        
        # Iterate over matches under this protein
        for match_id, match_data in protein_data.items():
            if match_id == "info":
                continue  # Skip the info entry
            
            # Create a match element under the protein
            match_elem = ET.SubElement(protein_elem, "match", 
                                       id=match_data["id"], 
                                       name=match_data["name"], 
                                       dbname=match_data["dbname"],
                                       status=match_data["status"], 
                                       model=match_data["model"],
                                       evd=match_data["evd"])
            

            # Create lcn elements for each location in the match
            for loc in match_data["locations"]:
                
                frag_str = ""

                if (loc["fragments"]):
                    frag_list = sorted(loc['fragments'].split(","), key=lambda x: (x.split("-")[2]))
                    frag_list = sorted(loc['fragments'].split(","), key=lambda x: (int(x.split("-")[0])))
                    frag_str = ','.join(frag_list)
                else:
                     frag_str = '-'.join([loc["start"], loc["end"], "S"])

                lcn_elem = ET.SubElement(match_elem, "lcn", 
                                         start=str(loc['start']), 
                                         end=str(loc['end']),
                                         fragments=frag_str,
                                         score=str(loc['score']))
    
    # Create an XML tree and write it to a file
    tree = ET.ElementTree(root)
    tree.write("output.xml", encoding="utf-8", xml_declaration=True)

    return grouped

def standardize():

    # Load the XML file
    tree = ET.parse("output.xml")
    root = tree.getroot()

    # Iterate over each protein and sort its matches
    for protein in root.findall('protein'):
        # Get all match elements
        matches = list(protein.findall('match'))

        # Sort matches by the 'score' attribute (convert score to integer for sorting)
        sorted_matches = sorted(matches, key=lambda x: x.get('id'))

        # Clear the original match elements from the protein
        for match in matches:
            protein.remove(match)

        # Append the sorted match elements back to the protein
        for match in sorted_matches:
            protein.append(match)

            locations = list(match.findall('lcn'))
            sorted_lcsn = sorted(locations, key=lambda x: int(x.get('start')))

                # Clear the original match elements from the protein
            for lcn in locations:
                match.remove(lcn)

        # Append the sorted match elements back to the protein
            for lcn in sorted_lcsn:
                match.append(lcn)

    # Save the modified XML back to a file
    tree.write("output.xml", encoding='utf-8', xml_declaration=True)

def get_differences(new_path, original_path):
    new = xmltodict.parse(open(new_path, "r").read(), attr_prefix = "", process_namespaces=True)
    original = xmltodict.parse(open(original_path, "r").read(), attr_prefix = "", process_namespaces=True)
    return new, original, DeepDiff(new, original)

In [165]:
un = 'interpro'
cs = 'ora-dlvm-119.ebi.ac.uk:1521/IPREAD'
pw = "olymp"

db = oracledb.connect(user=un, password=pw, dsn=cs)
cursor = db.cursor()

In [166]:
# Parse the XML file
tree = ET.parse("original.xml")
root = tree.getroot()

# Find all <lcn> elements and remove the 'representative' attribute if present
for lcn in root.findall('.//lcn'):
    if 'representative' in lcn.attrib:
        del lcn.attrib['representative']

# Find all <ipr> elements and remove them
# Iterate over a copy of the list of <ipr> elements to avoid modifying the list while iterating
for ipr in root.findall('.//ipr'):
    # Get the parent of <ipr> by iterating over the tree
    for parent in root.iter():
        if ipr in parent:
            parent.remove(ipr)
            break  # Exit the loop once the <ipr> element is removed
        
# Write the modified tree back to the XML file (or a new file)
tree.write("processed.xml", encoding="utf-8", xml_declaration=True)

In [167]:
tree = ET.parse("processed.xml") 
root = tree.getroot()

# Extract protein IDs
protein_ids = [protein.get("id") for protein in root.findall('protein')]
proteins_sql = ["\'" + protein  + "\'" for protein in protein_ids]

In [168]:
sql = f"""

WITH 

proteins AS (
    SELECT PROTEIN_AC, NAME, DBCODE, CRC64, LEN,
           TO_CHAR(TIMESTAMP, 'YYYY-MM-DD') AS TIMESTAMP,
           FRAGMENT, TO_CHAR(TAX_ID) AS TAX_ID
    FROM INTERPRO.PROTEIN
    WHERE PROTEIN_AC IN ({','.join(proteins_sql)})
    ORDER BY PROTEIN_AC
),

matches AS (
    -- Limit the number of rows from the MATCH and FEATURE_MATCH tables
    SELECT PROTEIN_AC, METHOD_AC, MODEL_AC, POS_FROM, POS_TO, FRAGMENTS, SCORE, DBCODE, EVIDENCE, STATUS
    FROM INTERPRO.MATCH
)

SELECT P.PROTEIN_AC, P.NAME, M.DBCODE, P.CRC64, P.LEN, P.TIMESTAMP, P.FRAGMENT, P.TAX_ID,
       M.METHOD_AC, M.MODEL_AC, M.POS_FROM, M.POS_TO, M.FRAGMENTS, M.SCORE, MN.DESCRIPTION, M.STATUS,
       DB.DBSHORT, CE.ABBREV

FROM proteins P
LEFT OUTER JOIN matches M
ON P.PROTEIN_AC = M.PROTEIN_AC

LEFT OUTER JOIN INTERPRO.METHOD MN
ON M.METHOD_AC = MN.METHOD_AC

LEFT OUTER JOIN INTERPRO.CV_DATABASE DB
ON M.DBCODE = DB.DBCODE

LEFT OUTER JOIN CV_EVIDENCE CE
ON M.EVIDENCE = CE.CODE

ORDER BY P.PROTEIN_AC

"""


In [169]:
protein_data = cursor.execute(sql)
protein_data = [[str(value) if value is not None else '' for value in row] for row in protein_data]

In [170]:
xml_data = create_xml(protein_data)

{'start': '48', 'end': '288', 'fragments': '48-288-S', 'score': '1.7e-111'}
{'start': '36', 'end': '378', 'fragments': '289-378-N,36-47-C', 'score': '1.7e-111'}
{'start': '12', 'end': '391', 'fragments': '12-391-S', 'score': 0}
{'start': '41', 'end': '381', 'fragments': '41-381-S', 'score': '1.7e-53'}
{'start': '9', 'end': '389', 'fragments': '9-389-S', 'score': '1.71e-98'}
{'start': '37', 'end': '385', 'fragments': '37-385-S', 'score': 0}
{'start': '37', 'end': '382', 'fragments': '37-382-S', 'score': '2.5e-108'}
{'start': '342', 'end': '565', 'fragments': '342-565-S', 'score': '1.18e-53'}
{'start': '17', 'end': '276', 'fragments': '17-276-S', 'score': '7.7e-05'}
{'start': '369', 'end': '550', 'fragments': '369-550-S', 'score': '1e-13'}
{'start': '341', 'end': '573', 'fragments': '341-573-S', 'score': '3.2e-62'}
{'start': '478', 'end': '492', 'fragments': '478-492-S', 'score': 0}
{'start': '361', 'end': '503', 'fragments': '361-503-S', 'score': '7.5e-19'}
{'start': '3', 'end': '300', 

In [171]:
standardize()

In [172]:
n, o, d = get_differences("processed.xml", "output.xml")

In [173]:
open("differences.json", "w").write(json.dumps(d))

2902